# Artificial Intelligence Nanodegree
## Recurrent Neural Network Project
## Project: Train a character level sequence generator

Welcome to the Recurrent Neural Network Project in the Artificial Intelligence Nanodegree! In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

## Getting started

In this project you will implement a popuular Recurrent Neural Network (RNN) architecture to create an English language sequence generator capable of building semi coherent english sentences from scratch by building them up character-by-character.  This will require a substantial amount amount of parameter tuning on a large trainnig corpus (at least 100,000 characters long).  In particular for this project we will be using a complete version of Sir Arthur Conan Doyl's classic book The Adventures of Sherlock Holmes.

The particular network architecture we will employ is known as  [Long Term Short Memory (LTSM)](https://en.wikipedia.org/wiki/Long_short-term_memory), which helps significantly avoid technical problems with optimization of RNNs.  

**Note:** Tuning RNNs is a computationally intensive endevour and thus timely on a typical CPU.  Using a reasonable sized Amazon GPU can speed up training by a factor of 10. 

In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

## Downloading and preprocessing a text dataset

Our first task is to grab a large text corpus for use in training, and on it we perform a several light of pre-processing tasks.  The default corpus we will use is Sherlock Holmes, but you can use a variety of others as well.

In [52]:
# grab text-based file from the web using keras's get_file command
# path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
# path = get_file('alice.txt', origin="http://www.umich.edu/~umfandsf/other/ebooks/alice30.txt")
path = get_file('sherlock.txt', origin="http://www.gutenberg.org/cache/epub/1661/pg1661.txt")

# read in the text, transforming everything to lower case
text = open(path).read().lower()

# remove all 'new line' tags and long blank spaces
text = text.replace('\n', ' ').replace('\r', '')
text = text.replace('     ', ' ').replace('\r', '')
text = text.replace('   ', ' ').replace('\r', '')
text = text.replace('  ', ' ').replace('\r', '')

# count the number of unique characters in the text
chars = sorted(list(set(text)))

# create dictionaries containing each character
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# print some of the text, as well as statistics
print ("this corpus has total length = " +  str(len(text)))
print ("this corpus has = " +  str(len(chars)) + " number of unique characters")
print ('The first 500 characters of the text are')
print ('------------------------------------------------------------')
print (text[0:500])
print ('------------------------------------------------------------')

this corpus has total length = 142562
this corpus has = 46 number of unique characters
The first 500 characters of the text are
------------------------------------------------------------
 alice's adventures in wonderland  lewis carroll  the millennium fulcrum edition 3.0  chapter i  down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought alice `without pictures or conversation?' so she was considering in her own mind (as well as she could, for the hot day ma
------------------------------------------------------------


## Cutting our text into sequences

Now we need to cut up the text into equal length sequences.  However it can certainly be the case that a word at the start or end of a sequence might get cut off, so in order to not lose this information we cut up the text in a simiilar manner to how images / audio are cut for classification - via *windowing*.  Imagine the entire text as one long string.  We slide a window of fixed length along the string from left to right - taking a step of a certain number of characters each time - and take a snapshot of whats in the window at each moment.

In [55]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
# print out what a few of the first segments look like
print (sentences[0])
print (sentences[1])

nb sequences: 47508
Vectorization...
 alice's adventures in wonderland  lewis
ice's adventures in wonderland  lewis ca


Next, we use Keras to quickly build a single hidden layer RNN - where our hidden layer consists of LTSM modules.

In [56]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


With our RNN build we can now train our model on the input text data.

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
47508/47508 [==============================] - 1188s - loss: 3.0738  

----- diversity: 0.2
----- Generating with seed: "nce?"' `thank you, it's a very interesti"
nce?"' `thank you, it's a very interestitotooootoootoot tooooootoooooototo tootootoot totootootooooooootot bootooootototoooo tootomotot bootooooootooto toooooototooooooo to tootootoototoototot tooootootootoot otoouto tootootoo tooooootooootot toot tooot bootototoo toototoooot tootoootoooto tooootootot botootoot to toototooo tooot tototoooootootog tooooootoootoo tootoototototoootoootoooototot totoooototototoooo, tootoooooototot totoototo

----- diversity: 0.5
----- Generating with seed: "nce?"' `thank you, it's a very interesti"
nce?"' `thank you, it's a very interestimo toout mot, bouo to, tootooo tumonto toom,moot oo boootootooboovoot oomoboot,ot t o,o, atobo, tob,onot boo botoy, totyo oo,o to bt motyt oototoomo tot,ooot t t oomton,ot ototot ot tt toto